# Introduction à PyTorch
La documentation de la bibliothèque PyTorch est [ici](https://pytorch.org/docs/1.12/ ). Il est également possible d'accéder à la documentation d'une fonction en tapant `help(torch.nom_de_la_fonction)` dans le terminal python de Spyder (exemple : `help(torch.matmul)` après avoir importé la biblitohèque PyTorch `import torch`).

### Créer un nouveau script python et copier/coller le code suivant :

In [1]:
import torch 

### Fonctionnalité "autograd" de PyTorch

Un `torch.tensor` est l'équivalent en PyTorch d'un `numpy.array` en Numpy : il s'agit d'un tableau multidimensionnel. 

In [2]:
#Exemple de création d'un tableau bidimensionnel
x = torch.tensor([[1, 2, 3],[4, 5 ,6], [7, 8 ,9], [10, 11, 12]])
print(x.shape)

torch.Size([4, 3])


La principale différence entre un `numpy.array` et un `torch.tensor` est le fait que le `torch.tensor` permet l'utilisation de la fonctionnalité *autograd* (option `requires_grad=True`). 

In [3]:
#Exemple de création d'un tableau bidimensionnel en activant l'autograd
x = torch.tensor([[1., 2., 3.],[4., 5. ,6.], [7., 8. ,9.], [10., 11., 12.]],requires_grad=True)
print(x)

tensor([[ 1.,  2.,  3.],
        [ 4.,  5.,  6.],
        [ 7.,  8.,  9.],
        [10., 11., 12.]], requires_grad=True)


Lorsque cette fonctionnalité est activée pour un `torch.tensor` $X$ , un graphe de calcul se crée et chaque opération faisant intervenir (directement ou indirectement) $X$ est ajoutée à ce graphe de calcul. Tout ce processus est transparent pour l'utilisateur. Ce processus de création d'un graphe de calcul correspond à l'étape de **propagation avant** vue en cours.  Lorsque la méthode `.backward()` d'une variable `torch.tensor` $y$ **scalaire** du graphe de calcul, est exécutée, l'étape de **rétropropagation** s'effectue et calcule la dérivée $\frac{\partial y}{\partial X}$. Le résultat est stockée dans le champs `x.grad`.

In [4]:
#Exemple d'utilisation de l'autograd avec des scalaires

# Création des tenseurs scalaires
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2.)
b = torch.tensor(3.)

print(x.grad) # None

# Construction du graphe de calcul (propagation avant)
z = w*x
y = z+b # y = 2*x + 3

# Calcul du gradient (rétropropagation)
y.backward(torch.tensor(1.))

# Affichage du gradient
print(x.grad)    # x.grad = 2 

None
tensor(2.)


**Dessiner (sur papier) le graphe de calcul de l'exemple précédent.** Dans ce graphe, vous remarquerez qu'il y a trois *feuilles* : le tenseur $x$, le tenseur $w$ et le tenseur $b$. Pour obtenir  $\frac{\partial y}{\partial w}$ et $\frac{\partial y}{\partial b}$ en plus de $\frac{\partial y}{\partial x}$, il suffit d'utiliser l'option `requires_grad=True` sur $w$ et $b$.

In [5]:
#Exemple d'utilisation de l'autograd avec des scalaires

# Création des tenseurs scalaires
x = torch.tensor(4., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

print(x.grad) # None
print(w.grad) # None
print(b.grad) # None

# Construction du graphe de calcul (propagation avant)
z = w*x
y = z+b

# Calcul des gradients (rétropropagation)
y.backward(torch.tensor(1.))

# Affichage des gradients
print(x.grad)    # x.grad = 2.
print(w.grad)    # w.grad = 4. 
print(b.grad)    # b.grad = 1. 

None
None
None
tensor(2.)
tensor(4.)
tensor(1.)


Ce mécanisme d'autograd fonctionne de la même manière lorsque les feuilles sont des tableaux multimensionnels plutôt que des scalaires.

In [6]:
#Exemple d'utilisation de l'autograd avec des tenseurs 2D

# Création des tenseurs
X = torch.tensor([[1., 2., 3.],[4., 5. ,6.], [7., 8. ,9.], [10., 11., 12.]],requires_grad=True) #tableau 4x3 
W = torch.tensor([[1., 2.],[4., 5.], [7., 8.]],requires_grad=True) #tableau 3x2 
b = torch.tensor([4., 5.], requires_grad=True) #vecteur de taille 2


# Construction du graphe de calcul (propagation avant)
z1 = X.matmul(W)
z2 = z1 + b
y = z2.sum()

# Calcul des gradients (rétropropagation)
y.backward(torch.tensor(1.))

# Affichage des gradients
print(X.grad)
print(W.grad)
print(b.grad)


tensor([[ 3.,  9., 15.],
        [ 3.,  9., 15.],
        [ 3.,  9., 15.],
        [ 3.,  9., 15.]])
tensor([[22., 22.],
        [26., 26.],
        [30., 30.]])
tensor([4., 4.])


**Dessiner (sur papier) le graphe de calcul de l'exemple précédent.** Quelle devrait-être les tailles des variables `X.grad`, `W.grad` et `b.grad` ? Vérifier leurs tailles dans le code (attribut `.shape` d'un tenseur).

Comme nous venons de la voir, la fonctionnalité *autograd* est une implémentation du théorème de dérivation d'une fonction composée. Pour s'en convaincre, prenons le cas de la composition de deux fonctions $y=g(Z)$ et $Z=f(X)$. Nous allons comparer deux utilisations de l'autograd :

Cas 1) Calculer directement $\frac{\partial y}{\partial X}$ avec l'autograd

Cas 2) Calculer manuellement $\frac{\partial y}{\partial Z}$ (variable `dy_dZ`) et fournir ce gradient à l'autograd (`Z.backward(dy_dZ)`) pour qu'il termine le calcule de $\frac{\partial y}{\partial X}$.

Les gradients obtenus avec le cas 1 et le cas 2 doivent être parfaitement identiques.

In [7]:
# CAS 1

# Création des tenseurs
X = torch.tensor([[1., 2., 3.],[4., 5. ,6.], [7., 8. ,9.], [10., 11., 12.]],requires_grad=True) #tableau 4x3 
W = torch.tensor([[1., 2.],[4., 5.], [7., 8.]],requires_grad=True) #tableau 3x2 

# Construction du graphe de calcul (propagation avant)
Z = X.matmul(W)
y = Z.sum()

y.backward(torch.tensor(1.))

print(X.grad)
# CAS 2

# Création des tenseurs
X = torch.tensor([[1., 2., 3.],[4., 5. ,6.], [7., 8. ,9.], [10., 11., 12.]],requires_grad=True) #tableau 4x3 
W = torch.tensor([[1., 2.],[4., 5.], [7., 8.]],requires_grad=True) #tableau 3x2 

# Construction du graphe de calcul (propagation avant)
Z = X.matmul(W)
y = Z.sum()

dy_dZ = torch.ones(Z.shape) #dérivée de la fonction sum
Z.backward(dy_dZ)

print(X.grad)

tensor([[ 3.,  9., 15.],
        [ 3.,  9., 15.],
        [ 3.,  9., 15.],
        [ 3.,  9., 15.]])
tensor([[ 3.,  9., 15.],
        [ 3.,  9., 15.],
        [ 3.,  9., 15.],
        [ 3.,  9., 15.]])
